In [1]:
# import urllib.request
from inscriptis import get_text
import glob
from lxml import html
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [99]:
companyList = glob.glob("/Users/sharontsao/Desktop/moelis/wetransfer-44076d/test10q/*/*.html")

In [59]:
filelist = glob.glob("/Users/sharontsao/Desktop/moelis/wetransfer-44076d/test10q/0001339947/*.html")

In [101]:
allCompany = {}

In [61]:
allFile = {}

In [102]:
for i in range(0, len(companyList)):
    with open(companyList[i],'r') as f:
        tree = html.fromstring(f.read())
        text = tree.text_content()
        allCompany[companyList[i]] = text

In [66]:
for i in range(0, len(filelist)):
    with open(filelist[i],'r') as f:
        tree = html.fromstring(f.read())
        text = tree.text_content()
        allFile[filelist[i]] = text

In [68]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    temp = doc.replace('\n', ' ')
    temp = temp.replace('\xa0', ' ')
    temp = temp.replace('\x94', ' ')
    temp = temp.replace('\x92', ' ')
    temp = temp.replace('\x97', ' ')
    temp = temp.replace('\x93', ' ')
    temp = temp.replace('\x99', ' ')
    stop_free = " ".join([i for i in temp.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    no_num = ''.join([i for i in normalized if not i.isdigit()])
    singleletter_free = ' '.join( [w for w in no_num.split() if len(w)>1] )
    no_sth = ' '.join( [w for w in singleletter_free.split() if w != 'th'] )
    no_ic = ' '.join( [w for w in no_sth.split() if w != 'ic'] )

    return no_ic



In [78]:
def NestedDictValues(d):
    for v in d.values():
        if isinstance(v, dict):
            yield from NestedDictValues(v)
        else:
            v = v.replace('\n', ' ')
            v = v.replace("–", ' ')
            v = v.replace("\uf06e", ' ')
            v = v.replace("•", ' ')
            #v = v.replace("th", ' ')
            #v = v.rep
            
            yield v

In [105]:
allCompanyText = list(NestedDictValues(allCompany))

In [79]:
allText = list(NestedDictValues(allFile))

In [106]:
doc_clean_all = [clean(doc).split() for doc in allCompanyText]  

In [80]:
doc_clean = [clean(doc).split() for doc in allText]  

In [82]:
len(doc_clean[0])

6938

In [ ]:
# this is only for Viacom

In [83]:
import gensim
from gensim import corpora

In [84]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

In [94]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word = dictionary, passes=50, minimum_probability=0)

In [95]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [96]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      59.087952        1       1  0.038972  0.0
1      40.912048        1       2 -0.038972  0.0, topic_info=     Category         Freq                  Term        Total  loglift  \
term                                                                     
203   Default  1743.000000               company  1743.000000  30.0000   
252   Default   443.000000             continued   443.000000  29.0000   
1670  Default   218.000000        contentsviacom   218.000000  28.0000   
794   Default   296.000000                 other   296.000000  27.0000   
1899  Default   159.000000              incnotes   159.000000  26.0000   
1107  Default  1698.000000                   tax  1698.000000  25.0000   
1535  Default   134.000000            analysisof   134.000000  24.0000   
1666  Default   134.000000  contentsmanagement’s   134.000000  23.0000   
1509  Default   814.000000              adjusted   814.000000  22.0000   
1812  Default   131.000000              expenses   131.000000  21.0000   
1994  Default   128.000000              millions   128.000000  20.0000   
1025  Default  1075.000000                senior  1075.000000  19.0000   
1562  Default   116.000000                   asu   116.000000  18.0000   
3488  Default   197.000000             endedjune   197.000000  17.0000   
72    Default   194.000000             amusement   194.000000  16.0000   
714   Default  2368.000000                 month  2368.000000  15.0000   
1559  Default   107.000000                assets   107.000000  14.0000   
783   Default  2094.000000             operating  2094.000000  13.0000   
1925  Default   104.000000             interests   104.000000  12.0000   
550   Default   928.000000                impact   928.000000  11.0000   
1738  Default   312.000000              discrete   312.000000  10.0000   
292   Default   463.000000             debenture   463.000000   9.0000   
463   Default   300.000000                fiscal   300.000000   8.0000   
1395  Default   537.000000                 point   537.000000   7.0000   
733   Default   172.000000              national   172.000000   6.0000   
1802  Default   167.000000             excluding   167.000000   5.0000   
1854  Default    89.000000                  free    89.000000   4.0000   
825   Default   587.000000            percentage   587.000000   3.0000   
479   Default   169.000000                former   169.000000   2.0000   
378   Default   140.000000            endedmarch   140.000000   1.0000   
...       ...          ...                   ...          ...      ...   
550    Topic2   493.915802                impact   928.665894   0.2624   
915    Topic2  1258.184937               quarter  3237.881348  -0.0515   
377    Topic2  1061.880737                 ended  2606.830078  -0.0043   
463    Topic2   208.834030                fiscal   300.312256   0.5305   
460    Topic2  1068.430786             financial  2657.112305  -0.0173   
739    Topic2  1004.842041                   net  2505.636230  -0.0200   
1395   Topic2   319.347870                 point   537.272583   0.3735   
565    Topic2   750.238464                income  1727.013916   0.0600   
997    Topic2  1154.363281               revenue  3148.912598  -0.1098   
747    Topic2   539.780457                  nine  1108.908813   0.1738   
991    Topic2   817.819336                result  1987.024658   0.0060   
240    Topic2   762.423218          consolidated  1875.947876  -0.0066   
292    Topic2   286.229828             debenture   463.519409   0.4117   
1057   Topic2   534.450500                   six  1161.261719   0.1177   
364    Topic2   662.036255              earnings  1608.308838   0.0061   
419    Topic2   794.558960               expense  2145.130371  -0.0994   
676    Topic2   551.899170                 march  1237.116577   0.0866   
386    Topic2   623.589783         entertainment  1545.455688

In [ ]:
# this is for all texts

In [107]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

In [110]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50, minimum_probability=0)

In [111]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [112]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      30.403429        1       1  0.007472  0.033662
0      26.355228        1       2 -0.035148  0.023334
2      25.232861        1       3 -0.046060 -0.039829
1      18.008486        1       4  0.073736 -0.017168, topic_info=      Category           Freq           Term          Total  loglift  logprob
term                                                                         
1872   Default  168205.000000        product  168205.000000  30.0000  30.0000
571    Default  104969.000000          could  104969.000000  29.0000  29.0000
1027   Default   94895.000000         fiscal   94895.000000  28.0000  28.0000
1494   Default  184463.000000            may  184463.000000  27.0000  27.0000
1580   Default   28111.000000        network   28111.000000  26.0000  26.0000
2209   Default  144204.000000        service  144204.000000  25.0000  25.0000
2556   Default   20282.000000           user   20282.000000  24.0000  24.0000
1521   Default  313836.000000        million  313836.000000  23.0000  23.0000
2323   Default   13213.000000     subscriber   13213.000000  22.0000  22.0000
4160   Default   12545.000000    programming   12545.000000  21.0000  21.0000
7355   Default   17356.000000  manufacturing   17356.000000  20.0000  20.0000
426    Default  227214.000000        company  227214.000000  19.0000  19.0000
2384   Default   12011.000000     television   12011.000000  18.0000  18.0000
79     Default   18698.000000    advertising   18698.000000  17.0000  17.0000
1161   Default    9560.000000          hotel    9560.000000  16.0000  16.0000
2257   Default   49394.000000       software   49394.000000  15.0000  15.0000
14150  Default   12084.000000  semiconductor   12084.000000  14.0000  14.0000
2462   Default   12011.000000         travel   12011.000000  13.0000  13.0000
846    Default  207681.000000          ended  207681.000000  12.0000  12.0000
171    Default   17733.000000        applied   17733.000000  11.0000  11.0000
6635   Default    8928.000000       echostar    8928.000000  10.0000  10.0000
4032   Default    9129.000000          cable    9129.000000   9.0000   9.0000
3036   Default   85506.000000      september   85506.000000   8.0000   8.0000
1475   Default   83315.000000          march   83315.000000   7.0000   7.0000
372    Default    9695.000000        charter    9695.000000   6.0000   6.0000
519    Default   64529.000000        content   64529.000000   5.0000   5.0000
507    Default  119936.000000   consolidated  119936.000000   4.0000   4.0000
1348   Default   91080.000000           june   91080.000000   3.0000   3.0000
4186   Default    7116.000000      satellite    7116.000000   2.0000   2.0000
608    Default  100442.000000       customer  100442.000000   1.0000   1.0000
...        ...            ...            ...            ...      ...      ...
426     Topic4   52337.992188        company  227214.500000   0.2462  -4.5263
1475    Topic4   25895.296875          march   83315.671875   0.5458  -5.2299
3036    Topic4   25688.976562      september   85506.164062   0.5118  -5.2379
846     Topic4   45144.906250          ended  207681.000000   0.1882  -4.6741
1348    Topic4   26496.949219           june   91080.281250   0.4796  -5.2069
507     Topic4   31506.785156   consolidated  119936.125000   0.3776  -5.0338
1291    Topic4   28396.732422       interest  102317.867188   0.4325  -5.1377
1545    Topic4   46001.433594          month  229713.781250   0.1062  -4.6553
1579    Topic4   41996.261719            net  198733.625000   0.1599  -4.7464
1214    Topic4   39181.527344         income  189387.312500   0.1387  -4.8158
1612    Topic4   30406.917969           note  133804.671875   0.2326  -5.0693
349     Topic4   36997.156250           cash  188822.453125   0.0844  -4.8731
1020    Topic4   38566.199219      financial  207111.500000   0.0334  -4.8316
1658    Topic4   26980.859375      operation 

In [ ]:
# import urllib.request
from inscriptis import get_text
import glob
from lxml import html
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

companyList = glob.glob("/Users/sharontsao/Desktop/moelis/wetransfer-44076d/test10q/*/*.html")

filelist = glob.glob("/Users/sharontsao/Desktop/moelis/wetransfer-44076d/test10q/0001339947/*.html")

allCompany = {}

allFile = {}

for i in range(0, len(companyList)):
    with open(companyList[i],'r') as f:
        tree = html.fromstring(f.read())
        text = tree.text_content()
        allCompany[companyList[i]] = text

for i in range(0, len(filelist)):
    with open(filelist[i],'r') as f:
        tree = html.fromstring(f.read())
        text = tree.text_content()
        allFile[filelist[i]] = text

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    temp = doc.replace('\n', ' ')
    temp = temp.replace('\xa0', ' ')
    temp = temp.replace('\x94', ' ')
    temp = temp.replace('\x92', ' ')
    temp = temp.replace('\x97', ' ')
    temp = temp.replace('\x93', ' ')
    temp = temp.replace('\x99', ' ')
    stop_free = " ".join([i for i in temp.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    no_num = ''.join([i for i in normalized if not i.isdigit()])
    singleletter_free = ' '.join( [w for w in no_num.split() if len(w)>1] )
    no_sth = ' '.join( [w for w in singleletter_free.split() if w != 'th'] )
    no_ic = ' '.join( [w for w in no_sth.split() if w != 'ic'] )

    return no_ic



def NestedDictValues(d):
    for v in d.values():
        if isinstance(v, dict):
            yield from NestedDictValues(v)
        else:
            v = v.replace('\n', ' ')
            v = v.replace("–", ' ')
            v = v.replace("\uf06e", ' ')
            v = v.replace("•", ' ')
            #v = v.replace("th", ' ')
            #v = v.rep
            
            yield v

allCompanyText = list(NestedDictValues(allCompany))

allText = list(NestedDictValues(allFile))

doc_clean_all = [clean(doc).split() for doc in allCompanyText]  

doc_clean = [clean(doc).split() for doc in allText]  

len(doc_clean[0])

# this is only for Viacom

import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

# this is for all texts

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)